Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

do this first
in the anaconda prompt  or in the Google Colab

pip install pandas-datareader
pip install yfinance

In [56]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np 
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [57]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".
start = dt.datetime(2020,9,22)
end = dt.datetime.now()

stock_symbol = ['AVAX-USD']

In [58]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.
deletd_column =[]
df = pdr.get_data_yahoo(stock_symbol, start, end).dropna()
df1 = df[["Open","Adj Close"]]

#df = #TODO
#df.head()
#df.shape
df1.columns
#df["Open"]

[*********************100%***********************]  1 of 1 completed


Index(['Open', 'Adj Close'], dtype='object')

In [59]:
df1

,Open,Adj Close
Date,,
2020-09-22,4.986754,5.234632
2020-09-23,5.234104,3.521812
2020-09-24,3.521812,4.578369
2020-09-25,4.585006,4.711471
2020-09-26,4.725395,4.541182
...,...,...
2021-11-05,79.047600,74.364738
2021-11-06,74.296654,78.685204
2021-11-07,78.805908,87.705421


In [60]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df1['Close_15_Rolling'] = df1['Adj Close'].rolling(16).mean() #start from zero ?

df1['Open_1_Change'] = df1['Open'].diff()

df1['Adj Close_Change'] = df1['Adj Close'].diff(-1)

df1=df1.dropna()
X=df1
X

<ipython-input-60-be726e1ccb4e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Close_15_Rolling'] = df1['Adj Close'].rolling(16).mean() #start from zero ?
<ipython-input-60-be726e1ccb4e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Open_1_Change'] = df1['Open'].diff()
<ipython-input-60-be726e1ccb4e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,Open,Adj Close,Close_15_Rolling,Open_1_Change,Adj Close_Change
Date,,,,,
2020-10-11,4.022435,4.095879,4.193274,-0.219528,-0.054177
2020-10-14,4.483894,4.150056,4.125488,0.461459,0.022703
2020-10-15,4.152928,4.127353,4.163335,-0.330966,0.086873
2020-10-16,4.127866,4.040480,4.129717,-0.025062,-0.155529
2020-10-17,4.038541,4.196009,4.097500,-0.089325,0.079947
...,...,...,...,...,...
2021-11-04,75.577637,79.293221,66.237581,7.804550,4.928482
2021-11-05,79.047600,74.364738,67.162887,3.469963,-4.320465
2021-11-06,74.296654,78.685204,68.248849,-4.750946,-9.020218


In [61]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
#np.where(a < 5, a, 10*a)
y = X["Adj Close_Change"]
y = np.where(y<0,-1,1)
y.shape
X.shape
X=X.drop("Adj Close_Change",axis="columns")
X

,Open,Adj Close,Close_15_Rolling,Open_1_Change
Date,,,,
2020-10-11,4.022435,4.095879,4.193274,-0.219528
2020-10-14,4.483894,4.150056,4.125488,0.461459
2020-10-15,4.152928,4.127353,4.163335,-0.330966
2020-10-16,4.127866,4.040480,4.129717,-0.025062
2020-10-17,4.038541,4.196009,4.097500,-0.089325
...,...,...,...,...
2021-11-04,75.577637,79.293221,66.237581,7.804550
2021-11-05,79.047600,74.364738,67.162887,3.469963
2021-11-06,74.296654,78.685204,68.248849,-4.750946


In [65]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

#y_data = X["Adj Close_Change"] 
#x_data = X.drop("Adj Close_Change", axis = 1)


#index = int(0.7*len(X))
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=0)

y_test.shape

(118,)

In [66]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.
logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)


#logistic = LogisticRegression()
#logistic = #TODO

In [67]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.04059965149472469]
1,Adj Close,[-0.035120785293993706]
2,Close_15_Rolling,[-0.0034666897081110397]
3,Open_1_Change,[-0.0040813308674150384]


In [69]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
probabilities

array([[0.53429132, 0.46570868],
       [0.53009255, 0.46990745],
       [0.53586013, 0.46413987],
       [0.53689186, 0.46310814],
       [0.50448981, 0.49551019],
       [0.52405575, 0.47594425],
       [0.56619989, 0.43380011],
       [0.52474134, 0.47525866],
       [0.45247399, 0.54752601],
       [0.54009181, 0.45990819],
       [0.53730284, 0.46269716],
       [0.53483639, 0.46516361],
       [0.54002276, 0.45997724],
       [0.53330066, 0.46669934],
       [0.53719319, 0.46280681],
       [0.46154204, 0.53845796],
       [0.53203116, 0.46796884],
       [0.53729991, 0.46270009],
       [0.54300494, 0.45699506],
       [0.52748558, 0.47251442],
       [0.55002174, 0.44997826],
       [0.48905596, 0.51094404],
       [0.53838186, 0.46161814],
       [0.52580525, 0.47419475],
       [0.53126308, 0.46873692],
       [0.53583139, 0.46416861],
       [0.48608137, 0.51391863],
       [0.54141525, 0.45858475],
       [0.53385276, 0.46614724],
       [0.53626776, 0.46373224],
       [0.

In [70]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

confusion_matrix =metrics.confusion_matrix(y_test,predictions)
confusion_matrix



array([[51, 10],
       [53,  4]], dtype=int64)

In [73]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test,predictions)
accuracy


0.4661016949152542

In [76]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X, y, cv=5)
cross_val

array([0.53164557, 0.46835443, 0.51282051, 0.37179487, 0.35897436])